In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("list.csv", encoding = "unicode_escape")

In [3]:
data['Txn Date'] = pd.to_datetime(data['Txn Date']) #to convert data from string to date format

In [4]:
data["Description"]=data['Description'].apply(lambda x:x.lower())

In [5]:
df1=data["Description"].value_counts()

In [6]:
import datetime as dt

In [7]:
data=data.drop(["Description"],axis=1)

In [8]:
Latest_Date = dt.datetime(2022,6,18)

In [9]:
data["Debit"]=data["Debit"].replace(",","",regex=True)
data["Debit"] = data["Debit"].astype(float)

In [10]:
RFMScores = data.groupby('category').agg({'Txn Date': lambda x: (Latest_Date - x.max()).days, 'category': lambda x: len(x), 'Debit':lambda x: x.sum()})

In [11]:
RFMScores["Txn Date"] = RFMScores["Txn Date"].astype(int)

In [12]:
RFMScores.rename(columns = {"Txn Date":"Recency","category":"Frequency","Debit":"Monetary"},inplace=True)

In [13]:
RFMScores.reset_index()

,category,Recency,Frequency,Monetary
0,bills,33,22,4293.50
1,food,23,11,4561.99
2,shopping,33,10,7537.00
3,travel,43,8,2333.46


In [14]:
quantiles = RFMScores.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()

In [15]:
def RScoring(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def FnMScoring(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [16]:
RFMScores['R'] = RFMScores['Recency'].apply(RScoring, args=('Recency',quantiles,))
RFMScores['F'] = RFMScores['Frequency'].apply(FnMScoring, args=('Frequency',quantiles,))
RFMScores['M'] = RFMScores['Monetary'].apply(FnMScoring, args=('Monetary',quantiles,))

In [17]:
RFMScores['RFMGroup'] = RFMScores.R.map(str) + RFMScores.F.map(str) + RFMScores.M.map(str)

#Calculate and Add RFMScore value column showing total sum of RFMGroup values
RFMScores['RFMScore'] = RFMScores[['R', 'F', 'M']].sum(axis = 1)

In [18]:
RFMSorted=RFMScores.sort_values(by=["RFMGroup"])

In [19]:
RFMSorted = RFMSorted.head(2)
RFM=list(RFMSorted.index)

In [20]:
df2=df1.head(5)

In [21]:
df2=list(df2.index)
mylist=RFM+df2

In [22]:
print(mylist)

['food', 'bills']
